In [1]:
#%pip install gensim

In [2]:
import bow
import w2v
import numpy as np

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ma0me\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ma0me\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ma0me\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ma0me\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
import pandas as pd

url = "https://raw.githubusercontent.com/ironhack-labs/project-nlp-challenge/refs/heads/main/dataset/data.csv"

# Read CSV file
df = pd.read_csv(url)

# Display the first few rows
df.head()

,label,title,text,subject,date
0,1,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,1,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,1,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,1,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
df.isnull().sum()

label      0
title      0
text       0
subject    0
date       0
dtype: int64

In [6]:
print("\nNumber of duplicate rows in training data:", df.duplicated().sum())


Number of duplicate rows in training data: 201


In [7]:
# Remove duplicate rows
df = df.drop_duplicates()

In [8]:
train_x, test_x, train_y, test_y = bow.split_train_test(df)

_, vectorizer = bow.create_features(train_x)
tokenized_train = bow.tokenize_texts(train_x, vectorizer)
tokenized_test = bow.tokenize_texts(test_x, vectorizer)

c:\Users\ma0me\.conda\envs\tnsr-gpu\lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'wa'] not in stop_words.
  warnings.warn(


In [9]:
import gensim
word2vec_model_google = gensim.models.KeyedVectors.load_word2vec_format(
    './archive/GoogleNews-vectors-negative300.bin', 
    binary=True
)

In [10]:
train_x_embed = w2v.embed_texts(tokenized_train, word2vec_model_google)
test_x_embed = w2v.embed_texts(tokenized_test, word2vec_model_google)

In [11]:
# the size of every text is different
train_x_embed.apply(lambda x: len(x))

text
Coming off from her latest record-breaking album and a performance at the GRAMMYs, singer/songwriter extraordinaire Adele decided to take some downtime and enjoy Disneyland with her partner Simon Konecki and son Angelo.Her security detail must have been fierce, and day guided by Disney s wonderful cast members, but as you can see in the picture below, she and her little family seemed to have the time of their lives strolling through the park and clearly buying up a lot of the merchandise.Something else was noticed in the photos, too. Adele s son appears to be enjoying his fun-filled day at the park dressed as  Anna  from Disney s mega-hit Frozen, complete with dress and ballet flats. It seems as if the singer is allowing her son to completely be himself and choose what he wants to like and enjoy.1. Adele let her son dress up as Anna from Frozen. Amazing. 2. Peter is smiling cuz Disneyland pic.twitter.com/NwWb4BSN8C  Marc (@MarcMonster) February 16, 2016This, of course, isn t the f

In [12]:
train_embeddings, test_embeddings = w2v.pool(
    tokenized_train,
    tokenized_test,
    word2vec_model_google
)

In [13]:
def embed_text_with_google_w2v(text: list[str], word2vec_model):
    word_vectors = [word2vec_model[word] for word in text if word in word2vec_model]
    
    # If no words in the model, return a zero vector
    if len(word_vectors) == 0:
        return [np.zeros(word2vec_model.vector_size)]  
    
    # Return the average vector
    return np.array(word_vectors, dtype=np.float32)


def apply_embeddings(texts: list[str], word2vec_model):
    embeddings = [embed_text_with_google_w2v(text, word2vec_model) for text in texts]
    return embeddings

In [14]:
train_x_embed = apply_embeddings(tokenized_train, word2vec_model_google)
test_x_embed = apply_embeddings(tokenized_test, word2vec_model_google)

In [15]:
def pad_embeddings(embeddings, max_length):
    """Pad or truncate embeddings to a fixed length"""
    # Get embedding dimension from the first non-empty embedding
    embedding_dim = embeddings[0].shape[1]
    
    # Initialize output array with zeros
    padded = np.zeros((len(embeddings), max_length, embedding_dim), dtype=np.float32)
    
    # Fill the array with actual embeddings
    for i, emb in enumerate(embeddings):
        if len(emb) > max_length:
            padded[i] = emb[:max_length]
        else:
            padded[i, :len(emb)] = emb
            
    return padded

In [16]:
padded_train_embeddings = pad_embeddings(train_x_embed, 15)
padded_test_embeddings = pad_embeddings(test_x_embed, 15)
padded_train_embeddings.shape, padded_test_embeddings.shape


((31792, 15, 300), (7949, 15, 300))

In [17]:
padded_train_embeddings.mean()

-0.0019272377

In [18]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, Concatenate, Dropout, Flatten, Embedding
import numpy as np


def create_embedding_matrix(word2vec_model, vectorizer):
    embedding_matrix = np.zeros((len(vectorizer.vocabulary_) + 1, 300))
    for word, idx in vectorizer.vocabulary_.items():
        if word in word2vec_model:
            embedding_matrix[idx] = word2vec_model[word]
    return embedding_matrix

def yoon_kim_cnn(
    vocab_size, 
    word2vec_model,
    vectorizer,
    embedding_dim=300, 
    max_sequence_length=50, 
    num_classes=len(set(train_y)), 
    filter_sizes=[3, 4, 5], 
    num_filters=100
):
    """
    Implementation of Yoon Kim's CNN with three channels:
    1. Static pre-trained word2vec
    2. Non-static (fine-tunable) pre-trained word2vec
    3. Random initialized embeddings
    """
    # Single input for token indices
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    
    # Create embedding matrix from Google's word2vec
    embedding_matrix = create_embedding_matrix(word2vec_model, vectorizer)
    
    # Channel 1: Static pre-trained word2vec (non-trainable)
    embedding_static = Embedding(vocab_size, embedding_dim,
                               weights=[embedding_matrix],
                               input_length=max_sequence_length,
                               trainable=False, # set this to False to keep the embeddings static
                               name='static_channel')(sequence_input)
    
    # Channel 2: Non-static pre-trained word2vec (trainable)
    embedding_non_static = Embedding(vocab_size, embedding_dim,
                                   weights=[embedding_matrix],
                                   input_length=max_sequence_length,
                                   trainable=True, # set this to True to fine-tune the embeddings
                                   name='non_static_channel')(sequence_input)
    
    # Channel 3: Random initialized embeddings (trainable)
    embedding_random = Embedding(vocab_size, embedding_dim,
                               input_length=max_sequence_length,
                               trainable=True,
                               name='random_channel')(sequence_input)
    
    # Convolutional blocks for each channel and filter size
    conv_blocks = []
    
    # Apply convolutions to each channel
    channels = [embedding_static, embedding_non_static, embedding_random]
    channel_names = ['static', 'non_static', 'random']
    
    # Functional API
    # different filters for each channel
    for i, (channel, name) in enumerate(zip(channels, channel_names)):
        for filter_size in filter_sizes:
            conv = Conv1D(filters=num_filters//3,
                         kernel_size=filter_size,
                         padding='valid', # no padding is added to the edges of the input sequence (is used in Yoon Kim's architecture)
                         activation='relu',
                         name=f'conv_{name}_{filter_size}',
                         strides=1)(channel)
            max_pool = MaxPooling1D(pool_size=max_sequence_length - filter_size + 1,
                                   name=f'maxpool_{name}_{filter_size}')(conv)
            conv_blocks.append(max_pool)
    
    # Concatenate all pooled features
    z = Concatenate()(conv_blocks)
    z = Flatten()(z)
    z = Dropout(0.5)(z)
    
    outputs = Dense(num_classes, activation='sigmoid')(z)
    
    # Create model
    model = Model(sequence_input, outputs)
    
    return model

In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# First, convert tokenized texts to sequences of vocabulary indices
def tokens_to_indices(tokenized_texts, vectorizer):
    token_sequences = []
    for text in tokenized_texts:
        sequence = [vectorizer.vocabulary_.get(word, 0) for word in text]
        token_sequences.append(sequence)
    return token_sequences

token_sequences = tokens_to_indices(tokenized_train, vectorizer)
test_token_sequences = tokens_to_indices(tokenized_test, vectorizer)

# Pad sequences to a fixed length
padded_train_sequences = pad_sequences(token_sequences, maxlen=50, padding='post', truncating='post')
padded_test_sequences = pad_sequences(test_token_sequences, maxlen=50, padding='post', truncating='post')

# Create and compile the model
model = yoon_kim_cnn(
    vocab_size=len(vectorizer.vocabulary_) + 1, 
    word2vec_model=word2vec_model_google, 
    vectorizer=vectorizer
)  # +1 for potential unknown tokens
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 static_channel (Embedding)     (None, 50, 300)      480300      ['input_1[0][0]']                
                                                                                                  
 non_static_channel (Embedding)  (None, 50, 300)     480300      ['input_1[0][0]']                
                                                                                                  
 random_channel (Embedding)     (None, 50, 300)      480300      ['input_1[0][0]']                
                                                                                              

In [20]:
# Convert labels to categorical format

train_y_cat = to_categorical(train_y)
test_y_cat = to_categorical(test_y)

# Train the model
model.fit(padded_train_sequences, train_y_cat, 
          validation_data=(padded_test_sequences, test_y_cat),
          epochs=10, batch_size=64)

Epoch 1/10
497/497 [==============================] - 18s 17ms/step - loss: 0.0526 - accuracy: 0.9849 - val_loss: 0.0146 - val_accuracy: 0.9974
Epoch 2/10
497/497 [==============================] - 5s 11ms/step - loss: 0.0108 - accuracy: 0.9978 - val_loss: 0.0103 - val_accuracy: 0.9976
Epoch 3/10
497/497 [==============================] - 5s 11ms/step - loss: 0.0058 - accuracy: 0.9987 - val_loss: 0.0112 - val_accuracy: 0.9979
Epoch 4/10
497/497 [==============================] - 5s 11ms/step - loss: 0.0025 - accuracy: 0.9994 - val_loss: 0.0124 - val_accuracy: 0.9977
Epoch 5/10
497/497 [==============================] - 5s 11ms/step - loss: 0.0014 - accuracy: 0.9997 - val_loss: 0.0142 - val_accuracy: 0.9977
Epoch 6/10
497/497 [==============================] - 5s 11ms/step - loss: 0.0016 - accuracy: 0.9996 - val_loss: 0.0115 - val_accuracy: 0.9979
Epoch 7/10
497/497 [==============================] - 5s 11ms/step - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.0192 - val_accuracy: 0.997

In [21]:
loss, accuracy = model.evaluate(padded_test_sequences, test_y_cat)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

249/249 [==============================] - 1s 3ms/step - loss: 0.0227 - accuracy: 0.9979
Test Accuracy: 99.79%


In [22]:
model.save("NLP_yoon_kim_cnn.h5")